In [15]:
get_ipython().system('pip install yfinance')
get_ipython().system('pip install PyPortfolioOpt')
get_ipython().system('pip install cvxopt')
get_ipython().system('pip install cvxpy')
import pypfopt
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import EfficientFrontier
from pypfopt import DiscreteAllocation

In [3]:
def covariance(prices):
    sample_cov = risk_models.sample_cov(prices, frequency=252)
    plotting.plot_covariance(sample_cov, plot_correlation=True)
    return sample_cov

from pypfopt import expected_returns
def returns(prices):
    mu = expected_returns.capm_return(prices)
    return mu

def min_variance(prices):
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(None, S, weight_bounds=(None, None))
    ef.min_volatility()
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return(weights)

def Max_Sharpe_port(prices):
    mu = returns(prices)
    S = covariance(prices)
    ef = EfficientFrontier(mu, S) 
    ef.max_sharpe()
    weights = ef.clean_weights()
    pd.Series(weights).plot.pie(figsize=(10,10));
    return weights

from pypfopt import objective_functions
risk = 0.15
def return_given_risk(prices, risk):
    mu = returns(prices)
    S = risk_models.sample_cov(prices)
    ef = EfficientFrontier(mu, S)
    ef.add_objective(objective_functions.L2_reg)  # gamme is the tuning parameter
    ef.efficient_risk(0.15)
    weights = ef.clean_weights()
    return weights


def minimize_risk(prices):
    mu = returns(prices)
    S = covariance(prices)
    ef = EfficientFrontier(mu, S, weight_bounds=(None, None))
    ef.add_objective(objective_functions.L2_reg)
    ef.efficient_return(target_return=0.07, market_neutral=True)
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return weights

In [4]:
# sorted by top AUM

tech = ['QQQ', 'VGT', 'XLK', 'ARKK', 'FDN'] 
healthcare = ['XLV', 'VHT','IBB','ARKG','IHI']
financials = ['XLF', 'VFH', 'KRE', 'KBE', 'FAS']
real_estate = ['VNQ', 'SCHH', 'VNQI', 'IYR', 'REET']
energy = ['XLE', 'ICLN', 'AMLP', 'VDE', 'XOP']
materials = ['GDX', 'XLB', 'GDXJ', 'GUNR', 'VAW']
consumer = ['XLY', 'XLC', 'VCR', 'PEJ', 'FXD']
industrials = ['XLI', 'VIS', 'JETS', 'ITA', 'IYT']
telecom = ['VOX', 'NXTG', 'FCOM', 'IYZ', 'IXP']
consumer_staples = ['XLP', 'VDC', 'IEV', 'FSTA', 'IYK']
utilities = ['XLU', 'VPU', 'IGF', 'NFRA', 'PAVE']

all_industries = ['QQQ', 'XLV', 'XLF', 'VNQ', 'XLE', 'GDX', 'XLY', 'XLI', 'VOX', 'XLP', 'XLU']

In [5]:
tickers = all_industries
ohlc = yf.download(tickers, period="max")
prices = ohlc["Adj Close"].dropna(how="all")

[*********************100%***********************]  11 of 11 completed


In [6]:
# Long/Short Min Variance
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
ef = EfficientFrontier(None, S, weight_bounds=(None, None))
ef.min_volatility()
weights = ef.clean_weights()
ef.portfolio_performance(verbose=True);
weights

Annual volatility: 13.0%


OrderedDict([('GDX', 0.08061),
             ('QQQ', -0.02869),
             ('VNQ', -0.00168),
             ('VOX', 0.3828),
             ('XLE', -0.07803),
             ('XLF', -0.17732),
             ('XLI', 0.02068),
             ('XLP', 0.47505),
             ('XLU', 0.11028),
             ('XLV', 0.24364),
             ('XLY', -0.02735)])

In [7]:
# Max Sharpe with Sector Constraints

mu = returns(prices)
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()

ef = EfficientFrontier(mu, S)  # weight_bounds automatically set to (0, 1)
ef.max_sharpe()
weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
weights

Expected annual return: 8.9%
Annual volatility: 17.0%
Sharpe Ratio: 0.41


OrderedDict([('GDX', 0.13811),
             ('QQQ', 0.0963),
             ('VNQ', 0.2363),
             ('VOX', 0.33193),
             ('XLE', 0.0),
             ('XLF', 0.0),
             ('XLI', 0.03495),
             ('XLP', 0.04316),
             ('XLU', 0.01126),
             ('XLV', 0.108),
             ('XLY', 0.0)])

In [8]:
from pypfopt import DiscreteAllocation

latest_prices = prices.iloc[-1]  # prices as of the day you are allocating
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=20000, short_ratio=0.3)
alloc, leftover = da.lp_portfolio()
print(f"Discrete allocation performed with ${leftover:.2f} leftover")
alloc

Discrete allocation performed with $27.91 leftover


{'GDX': 86,
 'QQQ': 6,
 'VNQ': 51,
 'VOX': 51,
 'XLI': 7,
 'XLP': 13,
 'XLU': 4,
 'XLV': 19}

In [9]:
get_ipython().system('pip install qiskit')
get_ipython().run_line_magic('matplotlib', 'inline')
from qiskit.finance import QiskitFinanceError
from qiskit.finance.data_providers import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime

     |████████████████████████████████| 8.5 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 17.6 MB 21.7 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 35.0 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 70.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 46.9 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 69.8 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 191 kB 64.7 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 133 kB 36.8 MB/s eta 0:00:01
     |████████████████████████████████| 582 kB 56.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 51.2 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 12.2 MB/s ta 0:00:01
     |████████████████████████████████

In [10]:
data = YahooDataProvider(
                 tickers = ["AEO", "ABBY", "AEP", "AAL"],
                 start=datetime.datetime(2018, 1, 1),
                 end=datetime.datetime(2018, 12, 31))
data.run()

num_assets = 4

mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

q = 0.5                   # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets      # set parameter to scale the budget penalty term

qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

In [11]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))


In [12]:
exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
result = exact_eigensolver.run()

print_result(result)

Optimal: selection [1 0 1 0], value -0.0004

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [1 0 1 0]	-0.0004		1.0000
 [1 1 1 1]	16.0036		0.0000
 [0 1 1 1]	4.0032		0.0000
 [1 0 1 1]	4.0019		0.0000
 [0 0 1 1]	0.0017		0.0000
 [1 1 0 1]	4.0039		0.0000
 [0 1 0 1]	0.0034		0.0000
 [1 0 0 1]	0.0022		0.0000
 [0 0 0 1]	4.0020		0.0000
 [1 1 1 0]	4.0011		0.0000
 [0 1 1 0]	0.0009		0.0000
 [0 0 1 0]	3.9997		0.0000
 [1 1 0 0]	0.0014		0.0000
 [0 1 0 0]	4.0012		0.0000
 [1 0 0 0]	3.9999		0.0000
 [0 0 0 0]	16.0000		0.0000


In [13]:
backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

print_result(result)

Optimal: selection [0. 1. 1. 0.], value 0.0009

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [0 1 1 0]	0.0009		0.6572
 [1 0 1 0]	-0.0004		0.2691
 [1 1 0 0]	0.0014		0.0733
 [0 0 1 1]	0.0017		0.0001
 [0 1 0 1]	0.0034		0.0001
 [0 0 0 1]	4.0020		0.0001
 [1 0 0 1]	0.0022		0.0001
 [1 1 0 1]	4.0039		0.0000
 [1 0 0 0]	3.9999		0.0000
 [0 1 1 1]	4.0032		0.0000
 [0 0 1 0]	3.9997		0.0000
 [0 0 0 0]	16.0000		0.0000
 [0 1 0 0]	4.0012		0.0000
 [1 1 1 1]	16.0036		0.0000
 [1 0 1 1]	4.0019		0.0000
 [1 1 1 0]	4.0011		0.0000


In [14]:
backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa = QAOA(qubitOp, cobyla, 3)

qaoa.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = qaoa.run(quantum_instance)

print_result(result)

Optimal: selection [1. 0. 1. 0.], value -0.0004

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [1 0 1 0]	-0.0004		0.1667
 [0 1 1 0]	0.0009		0.1667
 [1 1 0 0]	0.0014		0.1667
 [0 0 1 1]	0.0017		0.1667
 [1 0 0 1]	0.0022		0.1667
 [0 1 0 1]	0.0034		0.1666
 [1 0 0 0]	3.9999		0.0000
 [0 0 1 0]	3.9997		0.0000
 [0 1 0 0]	4.0012		0.0000
 [0 0 0 1]	4.0020		0.0000
 [1 1 1 0]	4.0011		0.0000
 [0 0 0 0]	16.0000		0.0000
 [1 0 1 1]	4.0019		0.0000
 [1 1 1 1]	16.0036		0.0000
 [0 1 1 1]	4.0032		0.0000
 [1 1 0 1]	4.0039		0.0000
